In [1]:
%pip install unidecode


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import pandas as pd
import spacy
from langdetect import detect, LangDetectException
from unidecode import unidecode
import string
import os
from io import BytesIO
import re

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    """Preprocesses the given text by cleaning and lemmatizing it."""
    text = text.lower().strip()
    text = unidecode(text)
    text = ''.join(char for char in text if not char.isdigit())
    text = ''.join(char for char in text if char not in string.punctuation)
    
    doc = nlp(text)
    filtered_tokens = [token.lemma_ for token in doc if not token.is_stop and len(token.lemma_) > 2 
                       and token.pos_ not in ['PRON', 'DET']]
    return ' '.join(filtered_tokens)

def detect_english(text):
    """Detects if the given text is English."""
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

def filter_english_text(df, text_column):
    """Filters the DataFrame to only include rows where the text is English."""
    df['is_english'] = df[text_column].apply(detect_english)
    filtered_df = df[df['is_english']].drop(columns=['is_english'])
    return filtered_df


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
def preprocessing_pipeline(df, text_column):
    """Applies text cleaning and English filtering to a DataFrame."""
    
    df = filter_english_text(df, text_column)
    
    if not df.empty:
        df['Processed Text'] = df[text_column].apply(preprocess_text)
    return df


In [ ]:
import os

def process_directory(directory_path, text_column):
    processed_dfs = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            processed_df = preprocessing_pipeline(df, text_column)
            processed_dfs.append(processed_df)
            
            processed_df.to_csv(f"../data/processed/processed_{filename}", index=False)
    return processed_dfs

